# Equations

$ V(s_t) = \gamma ^0r(s_t) + \gamma ^1r(s_{t+1}) + \gamma ^2r(s_{t+2}) + ... + \gamma ^nr(s_{t+n})$

$ \delta (s_t) = [r(s_t) + \gamma V(s_{t+1})] - V(s_t) $

$ V(s_t) = V(s_t) + \alpha\delta $

$ \pi (s_t) = argmax_a V(s_t) $

## SARSA

In [2]:
#from plotly.graph_objs import Scatter, Layout
import matplotlib.pyplot as plt
import plotly
import numpy as np
import random
import hrr
from plotly.graph_objs import Scatter, Layout, Surface
plotly.offline.init_notebook_mode(connected=True)

In [29]:
# state 2 is where the cheese is
# 1X3 maze

def reward(s):
    if s == 0:
        return 1
    else:
        return 0
    
def valuefunc(s,S,W,bias,a,A):
    return np.dot(hrr.convolve(S[s],A[a]),W)+bias
    
def policy(s,S,W,bias,a,A):
    x = random.random()
    ep = 0.01
    left = ((s-1)+len(S))%len(S)
    right = ((s+1)+len(S))%len(S)
    
    if x < ep or valuefunc(right,S,W,bias,a,A) == valuefunc(left,S,W,bias,a,A):
        c = random.choice([left,right])
        if c == left:
            return 0,left
        else:
            return 1,right
    elif valuefunc(right,S,W,bias,a,A) > valuefunc(left,S,W,bias,a,A):
        return 1,right
    else:
        return 0,left
        
def TD(v_size, num_ep, alpha, gamma): 
    #number of states
    nstates = v_size
    
    #length of hrr
    n = 128
    
    #encode each state as a unique hrr
    states = hrr.hrrs(n,nstates)
    
    #encode left and right action hrr
    #left = hrr.hrr(n)
    #right = hrr.hrr(n)
    actions = hrr.hrrs(n,2)
    
    #initialize weight vector with small random values
    W = np.random.random(n)*.01
    bias = 1 # start optimistically
    
    num_episodes = num_ep
    #V = [0]*v_size
    #V[9] = -10 # sudo barrier
    #print('Initial V is',V)
    #plt.plot([x for x in range(v_size)], V, 'bo',[x for x in range(v_size)], V, 'k')
    #plt.axis([0, v_size, 0, 1])
    #plt.show()
    for x in range(num_episodes):
        #print('Episode:',x)
        s = random.randrange(0,v_size)
        a = random.randrange(0,2)
        t_state = 0
        current_value = 0
        next_value = 0
        curr_sa = 0
        
        while s != t_state:
            new_action,new_state = policy(s,states,W,bias,a,actions) #get new state
            r = reward(s) #get reward at state s
            
            ## state action vector
            curr_sa = hrr.convolve(states[s],actions[a])
            next_sa = hrr.convolve(states[new_state],actions[new_action])
            
            ## Get the learned value V[s] of the current state
            ## Note this is a simple perceptron, or single-layer
            ## neural network which produces a weighted sum of the
            ## HRR for the current state.
            current_value = np.dot(curr_sa,W)+bias
            next_value = np.dot(next_sa,W)+bias
            #print("Value of V[",s,"] is",current_value)
            #print("Value of V[",new_state,"] is",next_value)
            #error = (r + gamma*V[new_state]) - V[s] #calc error
            error = (r + gamma*next_value)-current_value
            
            #V[s] += alpha*error #update V
            W += alpha*error*curr_sa
            #bias += alpha*error
            s = new_state #update state s
            a = new_action #update action a
           
        #reward absorbed
        r = reward(s)
        #print("final state is",s)
        #error = r - V[s]
        current_value = np.dot(curr_sa,W)+bias
        error = r - current_value
        #print("Error =",error)
        W += alpha*error*curr_sa
        #bias += alpha*error
        #print(list(map(lambda x: np.dot(x,W)+bias, states)))
        #V[s] += error
        ##############
        #print(V)
    V1 = list(map(lambda x: np.dot(hrr.convolve(x,actions[0]),W)+bias, states))
    V2 = list(map(lambda x: np.dot(hrr.convolve(x,actions[1]),W)+bias, states))
    data = [V1,V2]
    plotly.offline.iplot([
    dict(x=[x for x in range(len(V1))] , y=V1, type='scatter'),
    dict(x=[x for x in range(len(V1))] , y=V2, type='scatter')
    ])
    '''
    print('Final V is',V)
    plt.plot([x for x in range(v_size)], V, 'bo',[x for x in range(v_size)], V, 'k')
    plt.axis([0, v_size, 0, 1])
    plt.show()'''

In [30]:
TD(10,10000,.01,.5)

In [9]:
left = hrr.hrr(10)
right = hrr.hrr(10)
c = random.choice([left,right])
print(c)

[-0.38342166 -0.38684364  0.04789448  0.04221232  0.02771509 -0.38990873
 -0.19107692 -0.26906366  0.14885545 -0.12482073]
